## V2
# Autoencoder for image compression using Deep learning

<img src="https://www.oscprofessionals.com/wp-content/uploads/2019/11/lossless-compression-banner-2.jpg" height="100px" width="100%">

### By Tirtharaj Sinha([@tirtharajsinha](https://github.com/tirtharajsinha))


In [ ]:
# to subpress warning
import warnings
warnings.filterwarnings("ignore")

try:
    #importing librarys

    # cv2 is a python external package to do image processing and manipulation related stuff.
    import cv2 

    # imutils is a series of convenience functions to make basic image processing easy.
    import imutils

    # tensorflow is a python library for machine learning and artificial intelligence related work.
    import tensorflow as tf 


    # NumPy is a Python library used for working with arrays
    import numpy as np
    
    # pandas offers data structures and operations for manipulating numerical tables and time series.
    import pandas as pd

    # Keras is a library that provides a Python interface for artificial neural networks. 
    # Keras acts as an interface for the TensorFlow library.
    import keras


    # 1. Keras layers are the building blocks of the Keras library that can be stacked together for creating neural network models.
    # 2. Keras Conv2D creates a 2D convolution kernel that is wind with layers input which helps produce a tensor of outputs.
    # 3. maxpooling2D Downsamples the input along its spatial dimensions by taking the maximum value over an input window for each channel of the input. 
    # 4 .Flattening is converting the data into a 1-dimensional array for inputting it to the next layer. 
    # 5 .Dropout regularization technique for reducing overfitting in neural networks by preventing complex co-adaptations on training data.
    # 6. Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. 
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization


    #  Model groups layers into an object with training and inference features.
    from tensorflow.keras.models import Model

    # tensorflow.keras.callbacks is used to visualize training of a model.
    from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint 

    # used to split dataset(features and target) into test and test
    from sklearn.model_selection import train_test_split

    # F1/F Score is a measure of how accurate a model is by using Precision and Recall following the 
    # formula of: F1_Score = 2 * ((Precision * Recall) / (Precision + Recall)) 
    # Precision is commonly called positive predictive value.
    from sklearn.metrics import f1_score

    # shuffle the dataset for a even mixture of each type of feature and target.it gives better result.
    from sklearn.utils import shuffle

    # A one hot encoding allows the representation of categorical data to be more expressive.
    from sklearn.preprocessing import OneHotEncoder

    # confusion matrix is used to evaluate the accuracy of a classification.[[TP,FP],[FN,TN]]
    # A classification report is a performance evaluation metric in machine learning. 
    # It is used to show the precision, recall, F1 Score, and support of your trained classification model.
    from sklearn.metrics import confusion_matrix,classification_report

    # importing prebuild structure similarity index 
    from skimage.metrics import structural_similarity as SSIM
    
    # The OS module in Python provides functions for interacting with the operating system.
    import os

    # Matplotlib is a data visualization and graphical plotting library for Python.
    import matplotlib.pyplot as plt

    # seaborn is alse a data visualization and graphical plotting library for Python.
    import seaborn as sn

    # used to display markdown,image,control (frontend utilities)
    from IPython import display

    import time
    # time package
    
    from math import log10, sqrt
    # for mathematical operations
    
    import random
    
    
except:
    !pip install -r requirements.txt
# dataset path 
path = r"DATASET"
# make sure in this dir two folder named "yes" and "no" is present. 

In [ ]:
# GPU Info 
try:
    import GPUtil
except:
    !pip install GPUtil
    import GPUtil

    
# check physical computing devices
device=tf.config.experimental.list_physical_devices()
for i in device:
    print(i)
if len(device)>1:
    # find GPU details
    print("="*20, "GPU Details", "="*20)
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(gpu_id,gpu.name, gpu.driver,gpu.memoryTotal,gpu.temperature)



In [ ]:
# this function crop out the unnecessary part of a image.
# it takes two parameter -> image : numpy/cv2 image array , plot : binary (You want to plot after before effect of the image)


def image_threshholder(image, plot=False):
    # grayscalling the image
    # applying gausionBlur operation on grayscaled image.
    # binary threshholding the image to clean the gray scale range and makes the image binary colored based on range.
    # to remove thresh holding comment the next line
    grayscale=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred=cv2.GaussianBlur(grayscale,(5,5),0)
    new_image= cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY,cv2.CHAIN_APPROX_SIMPLE)[1]
    if plot:
        # plots the after before effect on the image based on parameter plot
        # plot if plot=true, ignore the if block if plot=False
        plt.figure()
        plt.figure(figsize=(12, 7))
        plt.subplot(1,2,1)
        plt.imshow(image)
        plt.tick_params(axis="both", which="both",
                        top=False,bottom=False,left=False,right=False,
                       labelbottom=False,labeltop=False,labelleft=False,
                       labelright=False)
        plt.title("(a)")
        
        plt.subplot(1,2,2)
        plt.imshow(grayscale,cmap='gray')
        plt.tick_params(axis="both", which="both",
                        top=False,bottom=False,left=False,right=False,
                       labelbottom=False,labeltop=False,labelleft=False,
                       labelright=False)
        plt.title("(b)")
        
#         plt.subplot(1,4,3)
#         plt.imshow(blurred,cmap='gray')
#         plt.tick_params(axis="both", which="both",
#                         top=False,bottom=False,left=False,right=False,
#                        labelbottom=False,labeltop=False,labelleft=False,
#                        labelright=False)
#         plt.title("(c)")
        
        
        
        
#         plt.subplot(1,4,4)
#         plt.imshow(new_image,cmap="gray")
#         plt.tick_params(axis="both", which="both",
#                         top=False,bottom=False,left=False,right=False,
#                        labelbottom=False,labeltop=False,labelleft=False,
#                        labelright=False)
#         plt.title("(d)")
#         plt.show()
        
    return grayscale,new_image

In [ ]:
# testing the crop_contour_brain_img() function
example_image=cv2.imread(path+"/yes/3.jpg")
img=image_threshholder(example_image,True)


In [ ]:
# view directory tree
def tree_printer(root):
    try:
        if ":" not in root:
            root=os.getcwd().replace("\\","/")+"/"+root
            print(root)
        if not os.path.isdir(root) and root!="":
            print(root,": path not exists....")
            return
    except:
        print("🛠️ set path of the data set from your local mechine")
        return
    
    
    print("🗁",root,"-->",len(os.listdir(os.path.join(root))), "Items present.")
    for name in os.listdir(root):
        try:
            print(" |- 🗁",name," "*2+"🏴",len(os.listdir(os.path.join(root, name))),"items")
        except:
            continue
            
tree_printer(path)

In [ ]:
image_size=(64,64) # defining the image size 
hotencoder = OneHotEncoder() # calling the OneHotEncoder
hotencoder.fit([[0], [1]]) # using binary crossentropy over catagorical crossentropy



# load the images from the local mechine and process before putting the resultent list.
def load_images(path,ishydro,type=["jpg","jpeg","png"],org=[],orgimages=[],thrsl=[],target=[]):

    for filepath in os.listdir(path):
        if filepath.split(".")[-1].lower() in type:
            
            img=cv2.imread(path+"/"+filepath)
               
            
            try:
                # filtering the image
                imgf,thrs=image_threshholder(img)
                # resizing the image in defined size
                imgf=cv2.resize(imgf,image_size)
                thrs=cv2.resize(thrs,image_size)
            except:
                print("Excluded image :",path+"/"+filepath)
                continue
            
            
            # 3d image to 2D
            imgf = np.expand_dims(imgf, 2)
            orgimg=np.reshape(imgf,image_size)
            
            thrs = np.expand_dims(thrs, 2)
            thrs=np.reshape(thrs,image_size)
            
            # pushing prepared data to list
            if ishydro:
                target.append(1)
            else:
                target.append(0)
            org.append(img)
            orgimages.append(orgimg)
            thrsl.append(thrs)
            
            
    return [org,orgimages,thrsl,target]

# plot images 
def show_image(datasets,num=4):  
    plt.figure()
    plt.figure(figsize=(7, 7))
    for i in range(num**2):
        
        plt.subplot(num, num, i+1)
        plt.imshow(datasets[i],cmap="gray")
        plt.axis('off')


In [ ]:
%%time
# load the image having brain cancer
rawimages,orgimages,thrsimages,target=load_images(path+"/yes",True,org=[],orgimages=[],thrsl=[],target=[])
img_index=list(range(len(target)))
high_i=len(target)
print(len(orgimages))
show_image(orgimages,num=3)




In [ ]:
rawimages1,orgimages1,thrsimages1,target1=load_images(path+"/no",False,org=[],orgimages=[],thrsl=[],target=[])
rawimages+=rawimages1
orgimages+=orgimages1
thrsimages+=thrsimages1
target+=target1
img_index+=list(range(high_i,len(target)))

# # print(images.shape)
show_image(orgimages[high_i:],num=3)
# print(result)

In [ ]:
# checking the dimention of the processed data
rawdata=np.array(rawimages)
orgdata=np.array(orgimages)
orgdata1=np.array(thrsimages)
target=np.array(target)
img_index=np.array(img_index)
print(orgdata1.shape)
# print(img_index)

## custom test-train splitter

In [ ]:
# customized test train splitter

def test_train_splitter(data,data1,train_ratio=0.2):
    data,data1=shuffle(data,data1)
    range=int(data.shape[0]*(1-train_ratio))
    x_train=data[:range]
    x_test_val=data[range:]
    x_val=x_test_val[:(x_test_val.shape[0]//2)]
    x_test=x_test_val[(x_test_val.shape[0]//2):]
    
    
    x_train1=data1[:range]
    x_test_val1=data1[range:]
    x_val1=x_test_val1[:(x_test_val1.shape[0]//2)]
    x_test1=x_test_val1[(x_test_val1.shape[0]//2):]
    
    return x_train,x_val,x_test,x_train1,x_val1,x_test1


# model starts here -----------:)

In [ ]:
# importing necessary packages
# view explanation in 2nd cell
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.datasets

## encoder model

In [ ]:
# encoder model func
# encoder takes image(4096bit) input of size 64X64
# encode the image in 64bit image(8X8)
def get_encoder():
    # Encoder input shape (4096,)
    x = tensorflow.keras.layers.Input(shape=(64,64), name="encoder_input")
    
    # flattening the data
    flat_x=tf.keras.layers.Flatten()(x)
    
    # scalling the data between 0..1
#     scaled_x=tf.keras.layers.Rescaling(1./255)(flat_x)
    
    # neurons in a Dense layer is now 2048 
    encoder_dense_layer1 = tensorflow.keras.layers.Dense(units=2048, name="encoder_dense_1")(flat_x)
    encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer1)

    # neurons in a Dense layer is decreased to 1024
    encoder_dense_layer2 = tensorflow.keras.layers.Dense(units=1024, name="encoder_dense_2")(encoder_activ_layer1)
    encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(encoder_dense_layer2)
    
    # neurons in a Dense layer is decreased to 512
    encoder_dense_layer3 = tensorflow.keras.layers.Dense(units=512, name="encoder_dense_3")(encoder_activ_layer2)
    encoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_3")(encoder_dense_layer3)
    
    # neurons in a Dense layer is decreased to 256
    encoder_dense_layer4 = tensorflow.keras.layers.Dense(units=256, name="encoder_dense_4")(encoder_activ_layer3)
    encoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_4")(encoder_dense_layer4)
    
    # neurons in a Dense layer is decreased to 64
    encoder_dense_layer5 = tensorflow.keras.layers.Dense(units=64, name="encoder_dense_5")(encoder_activ_layer4)
    encoder_activ_layer5 = tensorflow.keras.layers.LeakyReLU(name="encoder_output")(encoder_dense_layer5)

    # containerized the model
    encoder = tensorflow.keras.models.Model(x, encoder_activ_layer5, name="encoder_model")
    return encoder

encoder=get_encoder()
encoder.summary()

## decoder model

In [ ]:
# decoder model func
# encode the encoded image(64bit) back to original size(4096 bit).

def get_decoder():
    decoder_input = tensorflow.keras.layers.Input(shape=(64), name="decoder_input")

    # neurons in a Dense layer is now 256
    decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=256, name="encoder_dense_1")(decoder_input)
    decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer1)

    # neurons in a Dense layer is decreased to 512
    decoder_dense_layer2 = tensorflow.keras.layers.Dense(units=512, name="encoder_dense_2")(decoder_activ_layer1)
    decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_dense_layer2)
    
    # neurons in a Dense layer is decreased to 1024
    decoder_dense_layer3 = tensorflow.keras.layers.Dense(units=1024, name="encoder_dense_3")(decoder_activ_layer2)
    decoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_dense_layer3)
    
    # neurons in a Dense layer is decreased to 2048
    decoder_dense_layer4 = tensorflow.keras.layers.Dense(units=2048, name="encoder_dense_4")(decoder_activ_layer3)
    decoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_4")(decoder_dense_layer4)
    
    # neurons in a Dense layer is decreased to 4096
    decoder_dense_layer5 = tensorflow.keras.layers.Dense(units=4096, name="decoder_dense_5")(decoder_activ_layer4)
    decoder_activ_layer5 = tensorflow.keras.layers.LeakyReLU(name="decoder_output")(decoder_dense_layer5)
    
    # reshaped the image to 64X64
    decoder_output=tf.keras.layers.Reshape((64,64), input_shape=(4096,))(decoder_activ_layer5)
    
    decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
    
    return decoder

decoder=get_decoder()
decoder.summary()

## autoencoder model

In [ ]:
# Autoencoder model
# combine and sync the encoder and decoder model.
# parameter encoder model and decoder model.

def autoencoder(encoder,decoder):
    ae_input = tensorflow.keras.layers.Input(shape=(64,64), name="AE_input")
    
    # encoding the input images
    ae_encoder_output = encoder(ae_input)
    # decoding the encoded image back to original shape
    ae_decoder_output = decoder(ae_encoder_output)

    
    
    ae = tensorflow.keras.models.Model(ae_input, ae_decoder_output, name="AE")
    return ae


ae=autoencoder(encoder,decoder)
ae.summary()

    



In [ ]:
# calculating RMSE of autoencoder
# RMSE is a measure of how spread out these residuals are. 
# It tells you how concentrated the data is around the line of best fit. 
def rmse(y_true, y_predict):
    return tensorflow.keras.backend.mean(tensorflow.keras.backend.square(y_true-y_predict))

# AE Compilation
ae.compile(loss="mse", optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0005))



In [ ]:
# preparing brain tumor dataset spliting into test train default ratio is 0.2

x_train_index,x_val_index,x_test_index,x_train_org1,x_val_org1,x_test_org1=test_train_splitter(img_index,orgdata1)

print(x_train_index.shape,x_val_index.shape,x_test_index.shape)
print(x_train_org1.shape,x_val_org1.shape,x_test_org1.shape)
# print(x_test_index)



In [ ]:
%%time

# Training Autoencoder
ae.fit(x_train_org1, x_train_org1, epochs=20, batch_size=10, shuffle=True,validation_data=(x_val_org1, x_val_org1))


In [ ]:
%%time



start_time = time.time()

encoded_images = encoder.predict(x_test_org1)
encode_time=time.time() - start_time
decoded_images = decoder.predict(encoded_images)
# calculating execution time
execution_time=time.time() - start_time
decode_time=execution_time-encode_time

x_test_org=np.array([orgdata[i] for i in x_test_index])

# decoder original image resolution
org_res=(decoded_images.shape[0], 64, 64)

encoded_samples=np.reshape(encoded_images, newshape=(decoded_images.shape[0], 8, 8))

# reshaping the image of (4096,) to plotable image(64,64)
decoded_images_orig = np.reshape(decoded_images, newshape=org_res)
decoded_images_orig.shape

In [ ]:
# calculating RMSE of the trained model
rmse_ae=rmse(x_test_org,decoded_images)
print(rmse_ae)

In [ ]:
print("encoder image shape",encoded_images[0].shape)
print("decoder image shape",decoded_images[0].shape)

# ploting the comparition between original,encoded and decoded image.

plt.figure()
plt.figure(figsize=(12, 7))
plt.subplot(1,3,1)
sample_id=1
global_index=x_test_index[sample_id]

original_sample=x_test_org[sample_id]
plt.imshow(original_sample,cmap="gray")
plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
plt.title("original image")
        
plt.subplot(1,3,2)

encoded_sample = encoded_samples[sample_id]
plt.imshow(encoded_sample,cmap="gray")
plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
plt.title("encoded image")

plt.subplot(1,3,3)
decoded_sample=decoded_images_orig[sample_id]
plt.imshow(decoded_sample,cmap="gray")
plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
plt.title("decoded image")
plt.show()


### performance parameter
1. execution time
2. compression ratio
3. space saving
4. Bits per pixel (BPP)
5. Mean squared error (MSE)
6. Structure Similarity Index (SSIM)
7. Peak Signal to Noise Ratio (PSNR)
8. Percent rate of distortion (PRD)
9. Structural Content (SC)
10. Correlation Coefficient (CC)

In [ ]:
# avg execution time=total execution time/no of testing sample
print("Total execution time = {} seconds".format(execution_time))
print("Avarage Execution time = {} MilliSeconds".format(execution_time/encoded_images.shape[0]*1000))

print("Total encoding time = {} seconds".format(encode_time))
print("Avarage Encoding time = {} MilliSeconds".format(encode_time/encoded_images.shape[0]*1000))

print("Total decode Time = {} seconds".format(decode_time))
print("Avarage Decoding time = {} MilliSeconds".format(decode_time/encoded_images.shape[0]*1000))

print()




def performance_param_generator(encoded_sample,original_sample,imgnum="test"):
    if os.path.isdir(r"static"):
        for file in os.listdir(r'static'):
            if file.endswith('.jpg'):
                try:
                    os.remove(file)
                except:
                    pass
    else:
        os.system("mkdir static")
    
    # convert 2D image to 3D
    orgToSave=cv2.cvtColor(original_sample, cv2.COLOR_GRAY2RGB)
    
    # saving the image .jpg format.
    cv2.imwrite('static/comp_sample_'+str(imgnum)+'.jpg',encoded_sample)
    cv2.imwrite('static/org_sample_'+str(imgnum)+'.jpg',orgToSave)
    
    # getting the size of the image
    size_bytes_comp = os.path.getsize('static/comp_sample_'+str(imgnum)+'.jpg')
    size_bytes_org = os.path.getsize('static/org_sample_'+str(imgnum)+'.jpg')
    
    # number of pixel in compressed image for image sample 0
    comp_pixel=encoded_sample.shape[0]
    
    # converting the bytes to bit by multiplying by 8
    size_bits_comp=size_bytes_comp*8
    size_bits_org=size_bytes_org*8

    # Bits Per Pixel (BPP): BPP is defined as the ratio of the total size of the compressed image to the total number 
    # of the pixel in the image.
    BPP=size_bits_comp/comp_pixel


    # compression_ratio = orizinal image size / compressed image size
    compression_ratio=round(size_bits_org / size_bits_comp,3)


    # defined as the reduction in size relative to the uncompressed size:
    space_saving=round(1-(1/compression_ratio),2)

    
    return compression_ratio,BPP,space_saving,size_bits_comp,size_bits_org




In [ ]:
# Mean Square Error (MSE): MSE is the description of the cumulative squared error between the 
# compressed image and the original image
def MSE(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])*1000
    return round(err,3)




def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return round(psnr,3)



def PRD(original,decoded):
    sum_diff=np.sum((original - decoded))**2
    sum_org=np.sum(original)**2
    prd=sqrt(sum_diff/sum_org)*100
    return round(prd,3)




def SC(original,decoded):
    sum_org=np.sum(original**2)
    sum_dec=np.sum(decoded**2)
    sc=sum_org/sum_dec
    return round(sc,3)




def correlation_coeff(orgimage, decoded):
    cc = (np.sum(orgimage*decoded))/ ((sqrt((np.sum(orgimage)**2)) * (sqrt((np.sum(decoded)**2)))))
    return round(cc,4)


In [ ]:
comp_ratio,BPP,space_saving,size_bits_comp,size_bits_org=performance_param_generator(encoded_sample,original_sample)

print("File size of Compressed image :",size_bits_comp,"bits")
print("File size of Original image :",size_bits_org,"bits")
print("BPP for sample image :",BPP,"bits/pixel")
print("Compression ratio :", comp_ratio)
print("Space saving :",space_saving)

# calculating MSE of original and decompressed image
print("Mean Squared Error of original and decompressed image :",MSE(original_sample,decoded_sample))


# calculating SSIM of original and decompressed image

# Structure Similarity Index (SSIM): SSIM is used to measure the tendency of similarity between the 
# original image and the compressed image
print("Structure Similarity Index(SSIM) of original and decompressed image :",round(SSIM(original_sample,decoded_sample),3))

# Peak Signal to Noise Ratio (PSNR): PSNR is defined as the ratio of the maximum pixel intensity to the 
# mean square error
print("Peak Signal to Noise Ratio (PSNR) of original and decompressed image :",PSNR(original_sample,decoded_sample))

print("PRD of original and decompressed image :",PRD(original_sample,decoded_sample),"%")

print("Structural content of original and decompressed image :",SC(original_sample,decoded_sample))

print("Correlation Coefficient (CC) of original and decompressed image :",correlation_coeff(original_sample,decoded_sample))

In [ ]:
def performance_monitor(x_test_org,encoded_samples,decoded_images_orig,rawdata,x_test_index,num_images_to_show = 5):
    # comparing some more images between original and decode image.
    performance=[]
    
    if num_images_to_show==-1 or num_images_to_show>=x_test_org.shape[0]:
        num_images_to_show=x_test_org.shape[0]
        
#     suffled_index=random.sample(range(0,x_test_org.shape[0]), num_images_to_show)
    
    columns =['Id','target', 'encoded image size (bits)', 'original image size (bits)',"compression ratio","BPP","space saving","MSE","SSIM","PSNR","PRD","SC","CC"]
    font=14
    for rand_ind,_ in enumerate(x_test_org):
        plot_ind = rand_ind*5 + 1
        original_sample=x_test_org[rand_ind]
        
        plt.figure(figsize=(10, round(num_images_to_show*2)))
        
        plt.subplot(num_images_to_show, 5, plot_ind)
        plt.imshow(rawdata[x_test_index[rand_ind]], cmap="gray")
        plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
        if rand_ind==0:
            plt.title("Original",fontsize=font)
        
        plt.subplot(num_images_to_show, 5, plot_ind+1)
        plt.imshow(original_sample, cmap="gray")
        plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
        if rand_ind==0:
            plt.title("Grayscaled",fontsize=font)
        
        
        plt.subplot(num_images_to_show, 5, plot_ind+2)
        plt.imshow(original_sample, cmap="gray")
        plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
        if rand_ind==0:
            plt.title("Resized",fontsize=font)
        
        
        plt.subplot(num_images_to_show, 5, plot_ind+3)
        encoded_sample = encoded_samples[rand_ind]
        plt.imshow(encoded_sample, cmap="gray")
        plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
        if rand_ind==0:
            plt.title("Encoded",fontsize=font)
        
        
        
        decoded_sample=decoded_images_orig[rand_ind]
        # Clipping data to the valid range for imshow with RGB data ([0..1]
        plt.subplot(num_images_to_show, 5, plot_ind+4)
        plt.imshow(decoded_sample,cmap="gray")
        plt.tick_params(axis="both", which="both",
                top=False,bottom=False,left=False,right=False,
                labelbottom=False,labeltop=False,labelleft=False,
                labelright=False)
        if rand_ind==0:
            plt.title("Decoded",fontsize=font)
        
        
        comp_ratio,BPP,space_saving,size_bits_comp,size_bits_org=performance_param_generator(encoded_sample,original_sample,rand_ind)
        mse=MSE(original_sample,decoded_sample)
        ssim=round(SSIM(original_sample,decoded_sample),3)
        psnr=PSNR(original_sample,decoded_sample)
        prd=PRD(original_sample,decoded_sample)
        sc=SC(original_sample,decoded_sample)
        cc=correlation_coeff(original_sample,decoded_sample)
         
    
        performance.append([x_test_index[rand_ind],target[x_test_index[rand_ind]],size_bits_comp,size_bits_org,comp_ratio,BPP,space_saving,mse,ssim,psnr,prd,sc,cc])
        
    plt.figure()
    plt.scatter(encoded_images[:, 0], encoded_images[:, 1])
    plt.colorbar()
    table=pd.DataFrame(performance, columns=columns)
    table.set_index('Id',inplace=True)
    return table

In [ ]:
num_images_to_show=-1
table=performance_monitor(x_test_org,encoded_samples,decoded_images_orig,rawdata,x_test_index,num_images_to_show)


In [ ]:
liOfCR=table['compression ratio'].tolist()
avg_CR=sum(liOfCR)/len(liOfCR)
print("Average Compression Ratio is",avg_CR)
print("Minimum Compression Ratio is",min(liOfCR))
print("Maximum Compression Ratio is",max(liOfCR))
table.head(num_images_to_show)


# Applying model on single image

In [ ]:
def load_image(filepath,type=["jpg","jpeg","png"],orgimages=[],thrsl=[],rawimages=[]):
    if filepath.split(".")[-1].lower() in type:

        img=cv2.imread(filepath)
        rawimages.append(img)

        try:
            # filtering the image
            img,thrs=image_threshholder(img)
            # resizing the image in defined size
            img=cv2.resize(img,image_size)
            thrs=cv2.resize(thrs,image_size)
        except Exception as e:
            print(e)
            print("Excluded image :",filepath)
            


        # 3d image to 2D
        img = np.expand_dims(img, 2)
        orgimg=np.reshape(img,image_size)

        thrs = np.expand_dims(thrs, 2)
        thrs=np.reshape(thrs,image_size)

        # pushing prepared data to list
        orgimages.append(orgimg)
        thrsl.append(thrs)
            
            
    return [orgimages,thrsl,rawimages]

def pred_loose_img(img_relative_path=None,folder=None):
    orgimages_new=[]
    thrsimages_new=[]
    rawdata_new=[]
    if folder!=None:
        for file in os.listdir(path+folder):
            if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
                orgimages1,thrsimages1,rawdata1=load_image(path+folder+file,orgimages=[],thrsl=[],rawimages=[])
                orgimages_new+=orgimages1
                thrsimages_new+=thrsimages1
                rawdata_new+=rawdata1
#                 print(file)
    
    else:
        orgimages_new,thrsimages_new,rawdata_new=load_image(path+img_relative_path,orgimages=[],thrsl=[])
        
    orgdata_new=np.array(orgimages_new)
    orgdata1_new=np.array(thrsimages_new)
    rawdata_new=np.array(rawdata_new)



    encoded_image_s = encoder.predict(orgdata1_new)

    decoded_image_s = decoder.predict(encoded_image_s)
#     print(decoded_image_s.shape)


    # decoder original image resolution
    org_res=(decoded_image_s.shape[0], 64, 64)

    encoded_samples_s=np.reshape(encoded_image_s, newshape=(decoded_image_s.shape[0], 8, 8))

    # reshaping the image of (4096,) to plotable image(64,64)
    decoded_images_orig_s = np.reshape(decoded_image_s, newshape=org_res)
    x_test_index_s=np.array(list(range(decoded_image_s.shape[0])))
#     print(x_test_index_s)
    table=performance_monitor(orgdata_new,encoded_samples_s,decoded_images_orig_s,rawdata_new,x_test_index_s,-1)
    liOfCR=table['compression ratio'].tolist()
    avg_CR=sum(liOfCR)/len(liOfCR)
    print("Average Compression Ratio is",avg_CR)
    print("Minimum Compression Ratio is",min(liOfCR))
    print("Maximum Compression Ratio is",max(liOfCR))
    table.head()
    
    


In [ ]:
pred_loose_img(folder=r"/special_test/")

In [ ]:
pred_loose_img(img_relative_path=r"/yes/10.jpg")

# Comparison graph of various method based on performance parameter

In [ ]:
def graph_generator(data,color="black",marker="o"):
    ax=plt.subplot()
    ax.plot(data["method"], data["value"], color=color, marker=marker)
#     plt.title(data["Title"], fontsize=14)
    plt.xlabel(data["x_title"], fontsize=14)
    plt.ylabel(data["y_title"], fontsize=14)
    for i, v in enumerate(data["value"]):
        ax.annotate(str(v), xy=(i,v), xytext=(-7,7), textcoords='offset points')
#     plt.legend(handles=data["method"])
#     ax.grid(False)
    plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
    plt.ylim(min(data["value"])-1, max(data["value"])+2)
    plt.xlim(-1,7)
    plt.show()

In [ ]:
# compression ratio graph
cr_graph={
    "method":["LZW","Huffman coding","EZW","RLE","Shannon-Fano","Arithmetic coding","Proposed method"],
    "value":[6.319,4.203,3.664,3.576,3.982,4.205,9.422],
    "y_title":"Compression ratio",
    "x_title":"Method names",
    "Title":"Comparison graph of various method based on compression ratio"
}


psnr_graph={
    "method":["LZW","Huffman coding","EZW","RLE","Shannon-Fano","Arithmetic coding","Proposed method"],
    "value":[10.321,11.235,9.420,10.505,8.663,12.935,16.750],
    "y_title":"Peak Signal to Noise Ratio",
    "x_title":"Method names",
    "Title":"Comparison graph of various method based on PSNR"
}

snr_graph={
    "method":["LZW","Huffman coding","EZW","RLE","Shannon-Fano","Arithmetic coding","Proposed method"],
    "value":[16.7,18.1,17.7,17.0,16.5,17.2,18.6],
    "y_title":"Signal to Noise Ratio",
    "x_title":"Method names",
    "Title":"Comparison graph of various method based on SNR"
}

ssi_graph={
    "method":["LZW","Huffman coding","EZW","RLE","Shannon-Fano","Arithmetic coding","Proposed method"],
    "value":[0.092,0.157,0.296,0.350,0.199,0.236,0.070],
    "y_title":"Structure Similarity Index",
    "x_title":"Method names",
    "Title":"Comparison graph of various method based on SSI"
}

mse_graph={
    "method":["LZW","Huffman coding","EZW","RLE","Shannon-Fano","Arithmetic coding","Proposed method"],
    "value":[5.250,4.117,6.996,3.507,4.526,6.001,3.166],
    "y_title":"Mean Squred Error",
    "x_title":"Method names",
    "Title":"Comparison graph of various method based on MSE"
}

rmse_graph={
    "method":["LZW","Huffman coding","EZW","RLE","Shannon-Fano","Arithmetic coding","Proposed method"],
    "value":[2.291, 2.029, 2.645, 1.873, 2.127, 2.45, 1.779],
    "y_title":"Root Mean Squred Error",
    "x_title":"Method names",
    "Title":"Comparison graph of various method based on RMSE"
}



In [ ]:
graph_generator(cr_graph)
graph_generator(mse_graph,"red")
graph_generator(rmse_graph,"orange")
graph_generator(ssi_graph,"blue")
graph_generator(psnr_graph,"green")
graph_generator(snr_graph,"cyan")

In [ ]:
# !explorer .
# 70 train
# 15 validation 
# 15 testing

In [ ]:
a=[5.250,4.117,6.996,3.507,4.526,6.001,3.166]
for i in range(7):
    a[i]=round(sqrt(a[i]),3)
a

<h4 style="text-align:right;">Developed by: Tirtharaj Sinha</h4>
<h5 style="text-align:right;">github repository : <a href="https://github.com/tirtharajsinha/Autoencoder_for_image_compression">@tirtharajsinha/Autoencoder_for_image_compression</a></h5>